<a href="https://colab.research.google.com/github/avimistry-3/Assignment03/blob/main/Assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('brown')
nltk.download('movie_reviews')
nltk.download('stopwords')
from nltk.corpus import brown
from sklearn.pipeline import Pipeline
from nltk.corpus import movie_reviews
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords 
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import string
from scipy import spatial
import pandas as pd
import numpy as np

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Word2vec

In [ ]:
nltk.download('punkt')
data1=(brown.sents())
print(data1)
data2=(movie_reviews.sents())
print(data2)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]
[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]


In [ ]:
movie=[]
stop_words = set(stopwords.words('english')) 
for sent in movie_reviews.sents():
    sent= [w for w in sent if not w in stop_words]
    sent = ' '.join(sent)
    sent=sent.translate(str.maketrans('', '', string.punctuation))
    movie.append(sent)
for i in range(20):
  print(movie[i])
    
for i, sent in enumerate(movie):
    movie[i] = sent.split()


plot  two teen couples go church party  drink drive 
get accident 
one guys dies  girlfriend continues see life  nightmares 
 deal 
watch movie  sorta  find 


critique  mind  fuck movie teen generation touches cool idea  presents bad package 
makes review even harder one write  since generally applaud films attempt break mold  mess head  lost highway  memento   good bad ways making types films  folks  snag one correctly 
seem taken pretty neat concept  executed terribly 
problems movie 
well  main problem  simply jumbled 
starts  normal  downshifts  fantasy  world  audience member  idea  going 
dreams  characters coming back dead  others look like dead  strange apparitions  disappearances  looooot chase scenes  tons weird things happen  simply explained 
personally  mind trying unravel film every  give clue  get kind fed  film  biggest problem 
 obviously got big secret hide  seems want hide completely final five minutes 
make things entertaining  thrilling even engaging  meantime 
re

In [ ]:
brown=[]
stop_words = set(stopwords.words('english')) 
for sent in data1:
    sent= [w for w in sent if not w in stop_words]
    sent = ' '.join(sent)
    sent=sent.translate(str.maketrans('', '', string.punctuation))
    brown.append(sent) 
for i in range(10):
  print(brown[i])
for i, sent in enumerate(brown):
    brown[i] = sent.split()


The Fulton County Grand Jury said Friday investigation Atlantas recent primary election produced  evidence  irregularities took place 
The jury said termend presentments City Executive Committee  overall charge election   deserves praise thanks City Atlanta  manner election conducted 
The SeptemberOctober term jury charged Fulton Superior Court Judge Durwood Pye investigate reports possible  irregularities  hardfought primary Mayornominate Ivan Allen Jr 
 Only relative handful reports received   jury said   considering widespread interest election  number voters size city  
The jury said find many Georgias registration election laws  outmoded inadequate often ambiguous  
It recommended Fulton legislators act  laws studied revised end modernizing improving  
The grand jury commented number topics  among Atlanta Fulton County purchasing departments said  well operated follow generally accepted practices inure best interest governments  
Merger proposed
However  jury said believes  two of

In [ ]:
def processCorpus(train):
    traincorp = []
    for i in train:
        traincorp.append(word_tokenize(i))
    return traincorp

In [ ]:
train=str(brown)
traincorp = processCorpus(train)
modelW2V = Word2Vec(traincorp, min_count=1)
modelW2V.save('model1.bin')


TF-IDF

In [ ]:
brown=[]
for sent in data1:
    sent = ' '.join(sent)
    sent=sent.translate(str.maketrans('', '', string.punctuation))
    brown.append(sent)   
for i, sent in enumerate(brown):
    brown[i] = sent.split()

In [ ]:
total_doc_contain_word = Counter()
for doc in brown:
    count_words = Counter(doc)
    total_doc_contain_word.update(count_words)

In [ ]:
#Calculate TF for each word and doc
all_tf = []
for doc in brown:
    tf=[]
    total_words = len(doc)
    for word in total_doc_contain_word:
        if word not in doc:
            tf.append(0)
        else:
            tf.append(count_words[word] / total_words)
    all_tf.append(tf)

In [ ]:
#Calculate IDF for each word
idf = []
all_doc = len(brown)
for word in total_doc_contain_word:
    thisidf = np.log(all_doc / (total_doc_contain_word[word]+1))
    idf.append(thisidf)

In [ ]:
#Calculate TF-IDF 
doc_tfidf = []
for doc in all_tf:
    tf_idf = [a*b for a,b in zip(doc,idf)]
    doc_tfidf.append(tf_idf)

In [ ]:
#reshape tfidf matrix to make it word * doc
tfidf = np.array(doc_tfidf)
tfidf = [*zip(*tfidf)]
tfidf = np.array(tfidf)

In [ ]:
def nDCG(gold_set, model):
    evaluator = pytrec_eval.RelevanceEvaluator(
        gold_set, {'ndcg'})

    x = evaluator.evaluate(temp)
    item= []
    for i in x.items():
        item.append((i[1]['ndcg']))

In [ ]:
filepath= '/content/drive/MyDrive/SimLex-999.txt'